## Importar las librerías

In [1]:
# Importar las librerías

import numpy as np # Tratamiento numerico y matrices
import pandas as pd # Importar dataset y crear conjunto de entrenamiento y test
import torch
import torch.nn as nn # redes neuronales
import torch.nn.parallel # para crear cálculos en paralelo -> Optimización
import torch.optim as optim # Optimizador para minimizar nuestro error
import torch.utils.data
from torch.autograd import Variable

## Importar el dataset

In [2]:
# Importar el dataset

movies = pd.read_csv("ml-1m/movies.dat", sep = "::", header =None, engine = 'python', encoding = "latin-1")

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.shape

(3883, 3)

In [5]:
users = pd.read_csv("ml-1m/users.dat", sep = "::", header =None, engine = 'python', encoding = "latin-1")

In [6]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
users.shape

(6040, 5)

In [8]:
ratings = pd.read_csv("ml-1m/ratings.dat", sep = "::", header =None, engine = 'python', encoding = "latin-1")

In [9]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
ratings.shape

(1000209, 4)

## Preparar los conjuntos de entrenamiento y de testing

In [14]:
training_set = pd.read_csv( 'ml-100k/u1.base', sep ='\t', header = None)

In [15]:
training_set.shape

(80000, 4)

In [18]:
# Pytorch no acepta DataFrames sino arrays. Tenemos que reconvertirlo con numpy

training_set = np.array(training_set, dtype = "int") #no queremos que sea int64 sino int porque pytorch lo entiende mejor

In [19]:
test_set = pd.read_csv( 'ml-100k/u1.test', sep ='\t', header = None)
test_set = np.array(training_set, dtype = "int")

In [21]:
test_set.shape

(80000, 4)

In [ ]:
tes

## Obtener el número de usuarios y de películas

In [24]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
#Por que no sé si el usuario más alto está en el conjunto de test o de traing

In [25]:
nb_users

943

In [26]:
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [27]:
nb_movies

1682

## Convertir los datos en un array X[u,i] con usuarios u en fila y películas

In [30]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0]==id_user]
        id_ratings = data[:,2][data[:,0]==id_user]
        # Tb quiero un cero en las pelis que no ha clasificado -> Inicializo con 1682 ceros
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
        

In [31]:
training_set = convert(training_set)

In [33]:
test_set = convert(test_set)

## Convertir los datos a tensores de Torch

Un tensor es una matriz multidimensional de un solo tipo de dato

In [34]:
training_set= torch.FloatTensor(training_set)
test_set= torch.FloatTensor(test_set)

## ->Parte exclusiva Máquinas de Boltzmann (Paso 6)

## Convertir las valoraciones a valores binarios 1 (Me gusta) o 0 (No me gusta)

Hasta aquí los ceros no significan nada. Como queremos que el cero signifique "No me gusta", tenemos que cambiar los ceros actuales por otra cosa -> -1. los  -1 serán las pelis no vistas

In [36]:
# Pelis no vistas
training_set[training_set == 0] = -1

In [37]:
# transformamos las puntuaciones de 1 y 2 como no me gusta.Pytorch no acepta Os (or)
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0

In [38]:
# Las pelis con puntuaciones mayores o iguales a 3 ->Me gusta
training_set[training_set >= 3] = 1

In [39]:
# Hacemos lo mismo para el test_set
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Crear la arquitectura de la Red Neuronal (Modelo Probabilístico Gráfico)

Tenemos que crear una Clase que indique el número de nodos ocultos, los pesos de los mismos, los pesos de la probabilidad de un nodo visible, el sego (byas, el término independiente)...
Eato es la fórmula de los ai bi wij

In [47]:
class RBM():
    def __init__(self, nv, nh): #nv: nº de nodos visibles; nd: nº de nodos ocultos
        self.W= torch.randn(nh, nv) 
        #los pesos: aquí están las probabilidades que se asignan entre los nodos de capas visible y oculta
        #Los pesos se inician primero aleatoriamente con valores pequeños pero no nulos. Se distribuirán siguiendo una Normal estandar (medio cero, varianza 1)
        #El tensor tiene que ser de tamaño nh x nv para poder conectar la entrada de los nodos visibles con los ocultos
        self.a= torch.randn(1, nh) 
        #inicializamos el sesgo (término independiente). 
        #Hay que otorgar una dimensión más quer tiene que ver con la iteración -> Vector bidimensional
        self.b= torch.randn(1, nv)
        #definimos el sesgo específico para la capa de salida a través de los nodos visibles)- Vector bidimensional
    # 2ªfunción "de muestreo" que testeará los nodos de la capa oculta según las probabilidades (ph dado v) donde h es el nodo oculto y v el visible.
    # Y luego podremos comprobar que esos nodos se activarán o no en función de los valores de esa probabilidad
    def sample_h(self, x): # x -> valores de la capa visible (observaciones de los usuarios). No es v porque no tienen por qué ser todos
        wx = torch.mm(x, self.W.t()) #mini_batch size x nh
        # en torch hacemos así multiplicacion de dos matrices (tensores) como x es (1,nv) y W es (nh,nv) tengo que transponer W para poder hacer el producto
        # Añadimos la función de activación
        # wx -> ahora es (1, nh) -> igual que a. Así podemos sumarlos ahora. Pero si viene por bloques el vector a tiene que expandirse en la misma dimensión
        activation= wx + self.a.expand_as(wx) # combinación lineal de los pesos y sesgos
        p_h_given_v = torch.sigmoid(activation)    # aplicamos la función sigmoide al vector anterior. Generamos una muestra que active algunos de estos nodos
        return p_h_given_v, torch.bernoulli(p_h_given_v) # devolvemos una muestra de bernoulli de esta distribución construida a partir de las probabilidades
    # 3ª función: en base a la probabilidad de que se active un nodo visible muestrearemos qué nodos de la capa visible deben activarse
    def sample_v(self, y): # x -> valores de la capa oculta. No es h porque no tienen por qué ser todos
        wy = torch.mm(y, self.W) #  #mini_batch size x nv Aquí no hace falta transponer porque y es (1,nh) y W es (nh,nv)
        # en torch hacemos así multiplicacion de dos matrices (tensores)
        # Añadimos la función de activación
        # wy -> ahora es (1, nh) -> igual que a. Así podemos sumarlos ahora. Pero si viene por bloques el vector a tiene que expandirse en la misma dimensión
        activation= wy + self.b.expand_as(wy) # combinación lineal de los pesos y sesgos
        p_v_given_h = torch.sigmoid(activation)    #Probabilidad de activación de los nodos visibles conociendo los nodos ocultos
        return p_v_given_h, torch.bernoulli(p_v_given_h) # devolvemos una muestra de bernoulli de esta distribución construida a partir de las probabilidades
    # 4ª función:Divergencia Contrastante
    #Minimizar la función de energía  = maximizar el logaritmo de la probabilidad -> Lo aproximamos con la técnica de la divergencia contrastante
    def train(self,v0, vk, ph0, phk): #v0 ->el valor original vk, nodos visibles despues de k pasos, ph0-> probabilidades de la primera iteración de los nodos ocultos dados los valores de los visibles, phk-> probabilidad de los nodos ocultos después de k iteraciones (k divergencias contrastantes)
        self.W += (torch.mm(v0.t(),ph0) -torch.mm(vk.t(),phk)).t()
        self.b +=torch.sum((v0-vk),0)
        self.a +=torch.sum((ph0-phk),0)

In [48]:
nv = len(training_set[0])

In [49]:
nh = 100 # es un  parámetro que podemos elegir

In [50]:
batch_size = 100

In [51]:
rbm = RBM(nv, nh)

## Entrenar la RBM

In [52]:
nb_epoch= 10
for epoch in range (1, nb_epoch+1):
    training_loss = 0
    s= 0.
    for id_user in range(0,nb_users-batch_size, batch_size): 
        #v0 será el mismo (valoraciones originales de los usuarios) y vk irá cambiando a medida que hacemos el muestreo de Gibbs
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        #ph0 probabilidad de que un nodo oculto se active conociendo la entrada
        ph0,_ = rbm.sample_h(v0) 
        for k in range(10):
            #Hay que meter la divergencia contrastante en k pasos
            _,hk = rbm.sample_h(vk) # nos da lps nodos ocultos que se actiuvan en el paso k-ésimo
            _,vk = rbm.sample_v(hk) # nos calcula el nod visibles a partir del késimo nodo oculto -> actualiza vk
            # ¿Qué hacer con los -1? -> Lo mantenemos en -1
            vk[v0 <0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0>=0]-vk[vk>=0])) #Aquí sólo tengo encuenta las valoraciones en las que no hay -1
        s += 1.
    print("Epoch: "+str(epoch)+", Loss: "+str(training_loss/s)) # divido por s por que si no el error total sube con cada iteración


Epoch: 1, Loss: tensor(0.3697)
Epoch: 2, Loss: tensor(0.2519)
Epoch: 3, Loss: tensor(0.2511)
Epoch: 4, Loss: tensor(0.2599)
Epoch: 5, Loss: tensor(0.2485)
Epoch: 6, Loss: tensor(0.2485)
Epoch: 7, Loss: tensor(0.2452)
Epoch: 8, Loss: tensor(0.2475)
Epoch: 9, Loss: tensor(0.2483)
Epoch: 10, Loss: tensor(0.2429)


v0 = las valoraciones originales de ese batch de usuarios.
Es tu “verdad”.

vk = una copia inicial de v0, que luego se irá degradando, modificando, reconstruyendo durante el muestreo de Gibbs.

## Testear la RBM

In [55]:

testing_loss = 0
s= 0.
for id_user in range(nb_users): 

    v = training_set[id_user:id_user+1]# utilizamos conjunto de entrenamiento para activar neuronas
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0])>0:
 
 # Me basta un solo paso
 
        _,h = rbm.sample_h(v) 
        _,v = rbm.sample_v(h) 
      
        testing_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
        s += 1.
        print("Testing Loss: "+str(testing_loss/s))


Testing Loss: tensor(0.2889)
Testing Loss: tensor(0.2069)
Testing Loss: tensor(0.3165)
Testing Loss: tensor(0.2553)
Testing Loss: tensor(0.2723)
Testing Loss: tensor(0.2618)
Testing Loss: tensor(0.2626)
Testing Loss: tensor(0.2673)
Testing Loss: tensor(0.2746)
Testing Loss: tensor(0.2567)
Testing Loss: tensor(0.2533)
Testing Loss: tensor(0.2555)
Testing Loss: tensor(0.2619)
Testing Loss: tensor(0.2554)
Testing Loss: tensor(0.2606)
Testing Loss: tensor(0.2570)
Testing Loss: tensor(0.2573)
Testing Loss: tensor(0.2532)
Testing Loss: tensor(0.2556)
Testing Loss: tensor(0.2601)
Testing Loss: tensor(0.2663)
Testing Loss: tensor(0.2652)
Testing Loss: tensor(0.2626)
Testing Loss: tensor(0.2547)
Testing Loss: tensor(0.2465)
Testing Loss: tensor(0.2510)
Testing Loss: tensor(0.2491)
Testing Loss: tensor(0.2494)
Testing Loss: tensor(0.2468)
Testing Loss: tensor(0.2519)
Testing Loss: tensor(0.2494)
Testing Loss: tensor(0.2473)
Testing Loss: tensor(0.2485)
Testing Loss: tensor(0.2529)
Testing Loss: 

In [56]:
vt

tensor([[-1.,  1., -1.,  ..., -1., -1., -1.]])